In [2]:
import matplotlib.pyplot as plt
from lazypredict.Supervised import LazyClassifier, LazyRegressor
from sklearn.model_selection import train_test_split
import pandas as pd

In [3]:
#load data

train = pd.read_csv("C:/Users/Admin/OneDrive - National University of Singapore/NUS Related/Y3S1/BT4222/Project/cleaned_train.csv", na_filter = False)
test = pd.read_csv("C:/Users/Admin/OneDrive - National University of Singapore/NUS Related/Y3S1/BT4222/Project/cleaned_test.csv", na_filter = False)


In [4]:
y_train = train['price']
y_test = test['price']
print(y_train.shape)
print(y_test.shape)

(64447,)
(16113,)


In [5]:
train.drop('price', axis=1, inplace=True)
x_train = train
print(x_train.shape)
print(x_train.head())

(64447, 38)
   Unnamed: 0  index                                               name  \
0           0      0  Exclusive master bedroom w private bathroom n ...   
1           1      1          Newly Renovated Room B in Central Oakland   
2           2      2    Heart of San Diego in Old Town. Private Oasis 1   
3           3      3   Staylicious Modern Apartment super central in OC   
4           4      4  Luxury 3bd/2.5b spacious home, easy access to 101   

  description_sentiment neighborhood_overview_sentiment host_response_time  \
0               Neutral               Slightly Positive                 NA   
1               Neutral                              NA     within an hour   
2     Slightly Positive                         Neutral     within an hour   
3               Neutral               Slightly Positive     within an hour   
4               Neutral                              NA     within an hour   

  host_is_superhost  host_total_listings_count host_has_profile_pic 

In [6]:
test.drop('price', axis=1, inplace=True)
x_test = test
print(x_test.shape)
#print(x_test.head())

(16113, 38)


In [7]:
x_train.dtypes

Unnamed: 0                           int64
index                                int64
name                                object
description_sentiment               object
neighborhood_overview_sentiment     object
host_response_time                  object
host_is_superhost                   object
host_total_listings_count          float64
host_has_profile_pic                object
host_identity_verified              object
neighbourhood_cleansed              object
latitude                           float64
longitude                          float64
property_type                       object
room_type                           object
accommodates                         int64
bedrooms                           float64
beds                               float64
essentials                           int64
luxury                               int64
appliances                           int64
entertainment                        int64
security                             int64
comfort    

In [8]:
for t in x_train.columns:
    if x_train[t].dtypes == "object":
        x_train[t] = x_train[t].astype("category")
    
x_train.dtypes

Unnamed: 0                            int64
index                                 int64
name                               category
description_sentiment              category
neighborhood_overview_sentiment    category
host_response_time                 category
host_is_superhost                  category
host_total_listings_count           float64
host_has_profile_pic               category
host_identity_verified             category
neighbourhood_cleansed             category
latitude                            float64
longitude                           float64
property_type                      category
room_type                          category
accommodates                          int64
bedrooms                            float64
beds                                float64
essentials                            int64
luxury                                int64
appliances                            int64
entertainment                         int64
security                        

In [9]:
y_train.dtypes

dtype('float64')

In [10]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder

numeric_transformer = Pipeline(steps = [("simple_imputer", SimpleImputer(strategy='mean')), 
                                        ("one_hot_encoder", OneHotEncoder(handle_unknown = "ignore"))
                                       ])

categorical_transformer = Pipeline(steps = [("simple_imputer", SimpleImputer(strategy='most_frequent')), 
                                        ("one_hot_encoder", OneHotEncoder(handle_unknown = "ignore"))
                                           ])

preprocessor = ColumnTransformer(transformers = [("numeric_transformer", numeric_transformer, make_column_selector(dtype_exclude = 'category')),
                                                 ('categorical_transformer', categorical_transformer, make_column_selector(dtype_exclude = 'category'))], remainder = 'passthrough')

pipeline = Pipeline(steps = [
    ("preprocessor", preprocessor),
    ("gbr", gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=3, learning_rate=1.0, random_state=42))
])

pipeline.fit(x_train, y_train)


'\nfrom sklearn.svm import SVR\nfrom sklearn.pipeline import Pipeline\nfrom sklearn.impute import SimpleImputer\nfrom sklearn.preprocessing import OneHotEncoder, StandardScaler\nfrom sklearn.compose import ColumnTransformer, make_column_selector\nfrom sklearn.preprocessing import LabelEncoder\nfrom sklearn.preprocessing import OrdinalEncoder\n\nnumeric_transformer = Pipeline(steps = [("simple_imputer", SimpleImputer(strategy=\'mean\')), \n                                        ("one_hot_encoder", OneHotEncoder(handle_unknown = "ignore"))\n                                       ])\n\ncategorical_transformer = Pipeline(steps = [("simple_imputer", SimpleImputer(strategy=\'most_frequent\')), \n                                        ("one_hot_encoder", OneHotEncoder(handle_unknown = "ignore"))\n                                           ])\n\npreprocessor = ColumnTransformer(transformers = [("numeric_transformer", numeric_transformer, make_column_selector(dtype_exclude = \'category\')),\n

In [12]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from keras.models import Sequential
from keras.layers import Dense
 

ohe = OneHotEncoder()
ohe.fit(x_train)
x_train_enc = ohe.transform(x_train)

# prepare target
le = LabelEncoder()
le.fit(y_train)
y_train_enc = le.transform(y_train)

sc = StandardScaler()
X_train_std = sc.fit_transform(X_train)
X_test_std = sc.transform(X_test)

model = Sequential()
model.add(Dense(10, input_dim=x_train_enc.shape[1], activation='relu', kernel_initializer='he_normal'))
model.add(Dense(1, activation='linear'))
# compile the keras model
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
# fit the keras model on the dataset
model.fit(x_train_enc, y_train_enc, epochs=10, batch_size=8, verbose=2)
# evaluate the keras model
#_, accuracy = model.evaluate(X_test_enc, y_test_enc, verbose=0)
#print('Accuracy: %.2f' % (accuracy*100))

Epoch 1/10
8056/8056 - 207s - loss: 56234.5039 - accuracy: 3.2585e-04 - 207s/epoch - 26ms/step
Epoch 2/10
8056/8056 - 210s - loss: 35715.6875 - accuracy: 3.2585e-04 - 210s/epoch - 26ms/step
Epoch 3/10
8056/8056 - 212s - loss: 30489.8438 - accuracy: 3.2585e-04 - 212s/epoch - 26ms/step
Epoch 4/10
8056/8056 - 207s - loss: 27228.0332 - accuracy: 2.9482e-04 - 207s/epoch - 26ms/step
Epoch 5/10
8056/8056 - 205s - loss: 24937.6602 - accuracy: 2.9482e-04 - 205s/epoch - 25ms/step
Epoch 6/10
8056/8056 - 206s - loss: 23151.9590 - accuracy: 2.7930e-04 - 206s/epoch - 26ms/step
Epoch 7/10
8056/8056 - 206s - loss: 21666.5273 - accuracy: 2.7930e-04 - 206s/epoch - 26ms/step
Epoch 8/10
8056/8056 - 207s - loss: 20387.1895 - accuracy: 2.4827e-04 - 207s/epoch - 26ms/step
Epoch 9/10
8056/8056 - 208s - loss: 19204.3750 - accuracy: 2.4827e-04 - 208s/epoch - 26ms/step
Epoch 10/10
8056/8056 - 208s - loss: 18099.6621 - accuracy: 2.4827e-04 - 208s/epoch - 26ms/step


In [13]:
from sklearn.svm import SVR

clf = SVR(kernel='linear')
clf.fit(x_train_enc, y_train_enc)
clf.coef_

<1x293264 sparse matrix of type '<class 'numpy.float64'>'
	with 286780 stored elements in Compressed Sparse Row format>

In [25]:
from sklearn.inspection import permutation_importance
results = permutation_importance(clf, x_train_enc.toarray(), y_train_enc.toarray(), scoring='neg_mean_squared_error')
# get importance
importance = results.importances_mean
# summarize feature importance
for i,v in enumerate(importance):
    print('Feature: %0d, Score: %.5f' % (i,v))
# plot feature importance
#pyplot.bar([x for x in range(len(importance))], importance)
#pyplot.show()

MemoryError: Unable to allocate 141. GiB for an array with shape (64447, 293264) and data type float64

In [27]:
from sklearn.ensemble import GradientBoostingRegressor
import numpy as np
import matplotlib.pyplot as plt
from sklearn.inspection import permutation_importance

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=3, learning_rate=1.0, random_state=42)
gbrt.fit(x_train_enc, y_train_enc)

feature_importance = gbrt.feature_importances_
sorted_idx = np.argsort(feature_importance)
#sorted_idx
sorted_idx.shape

GradientBoostingRegressor(learning_rate=1.0, max_depth=2, n_estimators=3,
                          random_state=42)

(293264,)

In [24]:
# fit all models
#clf = LazyRegressor(predictions=True)
#models, predictions = clf.fit(x_train[:10000], x_test[:2000], y_train[:10000], y_test[:2000])

100%|██████████████████████████████████████████████████████████████████████████████████| 41/41 [03:48<00:00,  5.57s/it]


In [25]:
models

,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,


In [9]:
print(y_train)

0        45.00
1        59.00
2       185.00
3       105.00
4       255.00
         ...  
64442   171.00
64443    85.00
64444    60.00
64445   100.00
64446   328.00
Name: price, Length: 64447, dtype: float64
